In [ ]:
import os
import sys
sys.path.append("../lib/")
import glob
import multiprocessing
import cv2
from copy import deepcopy
import numpy as np
import random
import skimage
from skimage import io, transform
from skimage.metrics import structural_similarity as ssim
import itertools
from numpy.linalg import norm
from sewar.full_ref import ssim as sessim
from sewar.full_ref import uqi as uqi
from sewar.full_ref import msssim as msssim
from EvaluationUtils import *
from AttackSetting import *
from ImageMetrics import *

In [ ]:
def calculate_metrics_for_pair(current_attack_setting): 
    attack_result = AttackResult(current_attack_setting)
    
    previous = skimage.io.imread(current_attack_setting.previous)
    current = skimage.io.imread(current_attack_setting.current)

    # SSIM
    attack_result.ssim = ssim(previous, current, multichannel=True)

    # MS-SSIM
    attack_result.ms_ssim = float(msssim(previous, current))
    
    # L-Norm
    diff = (previous-current).flatten()/255.0
    attack_result.linfs = norm(diff.flatten(), np.inf)
    attack_result.l1 = np.sum(np.abs(diff))/diff.shape[0]
    attack_result.l2 = np.sqrt((diff.flatten()**2).sum()/diff.shape[0])

    # UQI            
    attack_result.uqi = uqi(previous, current)
    
    return attack_result

In [ ]:
def aggregate_metrics(camera, metrics, file_name):
    
    aggregated_image_metric = ImageMetrics(camera)
    
    # Distance between antenna and camera
    aggregated_image_metric.distance = metrics[0].distance
    # Frequency of the carrier wave
    aggregated_image_metric.fc = metrics[0].fc
    # Frequency of the sine wave/attack signal modulated onto the carrier
    aggregated_image_metric.wave_freq = metrics[0].wave_freq
    # Gain setting of the camera
    aggregated_image_metric.camera_gain = metrics[0].camera_gain
    # Gain setting of the USRP
    aggregated_image_metric.sdr_gain = metrics[0].sdr_gain
    
    ssim_values = []
    uqi_values = []
    linf_values = []
    l1_values = []
    l2_values = []
    ms_ssim_values = []
    
    for metric in metrics:
        ssim_values.append(metric.ssim)
        ms_ssim_values.append(metric.ms_ssim)
        linf_values.append(metric.linf)
        l1_values.append(metric.l1)
        l2_values.append(metric.l2)
        uqi_values.append(metric.uqi)

              
    # SSIM
    aggregated_image_metric.ssim_mean = np.mean(ssim_values)
    aggregated_image_metric.ssim_min = np.min(ssim_values)
    aggregated_image_metric.ssim_max = np.max(ssim_values)
    aggregated_image_metric.ssim_median = np.median(ssim_values)
    aggregated_image_metric.ssim_std = np.std(ssim_values)

    # MS-SSIM
    aggregated_image_metric.ms_ssim_mean = np.mean(ms_ssim_values)
    aggregated_image_metric.ms_ssim_min = np.min(ms_ssim_values)
    aggregated_image_metric.ms_ssim_max = np.max(ms_ssim_values)
    aggregated_image_metric.ms_ssim_median = np.median(ms_ssim_values)
    aggregated_image_metric.ms_ssim_std = np.std(ms_ssim_values)

    # Linf-Norm
    aggregated_image_metric.linf_mean = np.mean(linf_values)
    aggregated_image_metric.linf_min = np.min(linf_values)
    aggregated_image_metric.linf_max = np.max(linf_values)
    aggregated_image_metric.linf_median = np.median(linf_values)
    aggregated_image_metric.linf_std = np.std(linf_values)

    # L1-Norm
    aggregated_image_metric.l1_mean = np.mean(l1_values)
    aggregated_image_metric.l1_min = np.min(l1_values)
    aggregated_image_metric.l1_max = np.max(l1_values)
    aggregated_image_metric.l1_median = np.median(l1_values)
    aggregated_image_metric.l1_std = np.std(l1_values)

    # L2-Norm
    aggregated_image_metric.l2_mean = np.mean(l2_values)
    aggregated_image_metric.l2_min = np.min(l2_values)
    aggregated_image_metric.l2_max = np.max(l2_values)
    aggregated_image_metric.l2_median = np.median(l2_values)
    aggregated_image_metric.l2_std = np.std(l2_values)

    # UQI
    aggregated_image_metric.uqi_mean = np.mean(uqi_values)
    aggregated_image_metric.uqi_min = np.min(uqi_values)
    aggregated_image_metric.uqi_max = np.max(uqi_values)
    aggregated_image_metric.uqi_median = np.median(uqi_values)
    aggregated_image_metric.uqi_std = np.std(uqi_values)
    
    # Store the number of frame pairs that were aggregated
    aggregated_image_metric.values = len(uqi_values)
    
    EvaluationUtils.save_results(file_name, [aggregated_image_metric])

In [ ]:
def calculate_metrics_between_pairs(attack_settings, pairs, file_name, store_intermediate_results=True, file_name_intermediate_results="/home/code/data/intermediate_image_metrics.csv"):
    metrics_results = []
    # For each pair of frames run the calculation
    for pair in pairs:
        # For each frame pair create a copy of the attack settings
        current_attack_setting = deepcopy(attack_settings)
        current_attack_setting.previous = pair[0]
        current_attack_setting.current = pair[1]
        
        # Run the calculation for the current attack setting
        attack_result = calculate_metrics_for_pair(current_attack_setting)
        # Add the result to the list
        metrics_results.append(attack_result)

    # If activated, the results for each frame pair are stored 
    if store_intermediate_results:
        EvaluationUtils.save_results(file_name_intermediate_results, metrics_results)
    
    if len(metrics_results) > 0:
        # Aggregate the results stored in the list
        aggregate_metrics(attack_settings.camera, metrics_results, file_name)

In [ ]:
def run_calculation_job(parameters):

    # Since the calculation is running in separate processes,
    # the data is passed to the function via an array
    camera = parameters[0]
    distance = parameters[1]
    fc = parameters[2]
    wave_freq = parameters[3]
    file_name = parameters[4]
    camera_gains = parameters[5]
    sdr_gains = parameters[6]

    # For each camera gain run the calculation
    for camera_gain in camera_gains:
        # Iterate over different SDR gain values to see how it affects the distortions
        for sdr_gain in sdr_gains:
            # Create an instance that stores all parameters about the current attack settings
            attack_setting = AttackSetting(camera, distance, fc, wave_freq, camera_gain, sdr_gain)
            
            # Check if the image metrics for the current attack settings have already been calculated
            # If not, continue and calculate them
            if not EvaluationUtils.check_if_image_metrics_already_calculated(file_name, attack_setting):
                path = f"/home/code/data/frequency_sweep/{camera}/{distance}/{fc}MHz/{wave_freq}kHz/{camera_gain}/{sdr_gain}"            
                legitimate_frames, malicious_frames = EvaluationUtils.get_all_frames_at(path)

                # Get all combinations between the different frames
                legitimate_legitimate_pairs = list(itertools.combinations(legitimate_frames, r=2))
                legitimate_malicious_pairs = list(itertools.product(legitimate_frames, malicious_frames))
                
                # Calculate the image metrics betweent he frame pairs generated above
                calculate_metrics_between_pairs(attack_setting, legitimate_malicious_pairs, file_name)
                
                # Create an attack setting for legitimate frames, i.e., no attack signal was present during the capture
                attack_setting = AttackSetting(camera, 0, 0, 0, camera_gain, 0)
                # Check if for this attack setting the calculations have already been done
                if not EvaluationUtils.check_if_image_metrics_already_calculated(file_name, attack_setting):
                    calculate_metrics_between_pairs(attack_setting, legitimate_legitimate_pairs, file_name)

In [ ]:
def create_tasks_for_camera(camera):
    # This list contains the tasks that have to be calculated
    list_of_tasks = []
    # Path where the results will be stored
    results_path = "/home/code/data/image_metrics.csv"
    
    # Specify the number of CPU cores to use multiprocessing
    number_of_cpu_cores = 8
    
    distance = 3
    wave_freq = 1
    camera_gains = [29]
    sdr_gains = [30]
    
    # Set where the frequency sweep starts (in MHz)
    start_freq = 50
    # Set where the frequency sweep stops (in MHz)
    stop_freq = 5001
    
    # Iterate over all frequencies
    for fc in range(start_freq, stop_freq):
        # Create a task and add it to the list
        list_of_tasks.append([camera, distance, fc, wave_freq, results_path, camera_gains, sdr_gains])

    # Shuffle tasks
    random.shuffle(list_of_tasks)
    # Each task will be executed in a separate process
    p = multiprocessing.Pool(number_of_cpu_cores)
    p.map(run_calculation_job, list_of_tasks)

In [ ]:
create_tasks_for_camera("DFM25G445")